importacion de la liberia

In [4]:
import pandas as pd
import chardet 

Analisis y carga del archivo, revision de la codificacion

In [8]:
file = r"pandas_selector/file.csv"
def detect_encoding(file_path): 
    with open(file_path, 'rb') as file: 
        detector = chardet.universaldetector.UniversalDetector() 
        for line in file: 
            detector.feed(line) 
            if detector.done: 
                break
        detector.close() 
    return detector.result['encoding'] 

encoding = detect_encoding(file) 
print(f'The encoding of the file is: {encoding}') 

The encoding of the file is: ISO-8859-1


In [ ]:
df = pd.read_csv(file, na_filter=False, encoding='ISO-8859-1')
df.head(2)